In [1]:
import numpy as np
import pandas as pd

In [2]:
working_dir = '~/Project/pred-future-sales'
sales_train = pd.read_csv(working_dir + '/data/interim/sales_train.csv')
train_test = pd.read_csv(working_dir + '/data/interim/train_test.csv')

In [ ]:
# sales_train = sales_train.astype({'date': 'datetime64', 'date_block_num': 'int32', 'shop_id': 'int32', 
#                          'item_id': 'int32', 'item_price': 'float64', 'item_cnt_day': 'float64'})
# train_test = train_test.astype({'date_block_num': 'int32', 'shop_id': 'int32', 
#                          'item_id': 'int32', 'item_cnt_month': 'float64', 
#                          'city_code': 'int32', 'item_category_id': 'int32', 'type_code': 'int32'}, 
#                          'subtype_code': 'int32')

In [3]:
def lag_feature(df, lags, col):
    tmp = df[['date_block_num', 'shop_id', 'item_id', col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['date_block_num', 'shop_id', 'item_id', col + '_lag_' + str(i)]
        shifted['date_block_num'] += i
        df = pd.merge(df, shifted, on=['date_block_num', 'shop_id', 'item_id'], how='left')
    return df

In [4]:
train_test = lag_feature(train_test, [1, 2, 3, 6, 12], 'item_cnt_month')

In [6]:
def average_lag(df, group_keys, lag_nums, feature_names):
    size = len(group_keys)
    for i in range(size):
        group = df.groupby(group_keys[i]).agg({'item_cnt_month': ['mean']})
        group.columns = [feature_names[i]]
        group.reset_index(inplace=True)
        df = pd.merge(df, group, on=group_keys[i], how='left')
        df = lag_feature(df, lag_nums[i], feature_names[i])
        df.drop([feature_names[i]], axis=1, inplace=True)
    return df  
group_keys = [['date_block_num'], ['date_block_num', 'item_id'], ['date_block_num', 'shop_id'], \
                ['date_block_num', 'item_category_id'], ['date_block_num', 'type_code'], ['date_block_num', 'subtype_code'], \
                ['date_block_num', 'city_code'], ['date_block_num', 'shop_id', 'item_category_id'], ['date_block_num', 'shop_id', 'type_code'], \
                ['date_block_num', 'shop_id', 'subtype_code'], ['date_block_num', 'item_id', 'city_code']\
             ]
lag_nums = [ [1], [1, 2, 3, 6, 12], [1, 2, 3, 6, 12], \
            [1, 2, 3, 6, 12], [1], [1], \
            [1], [1], [1], \
            [1], [1]\
           ]
feature_names = ['item_cnt_month_avg', 'item_cnt_month_item_avg', 'item_cnt_month_shop_avg', \
                 'item_cnt_month_category_avg', 'item_cnt_month_type_avg', 'item_cnt_month_subtype_avg',\
                 'item_cnt_month_city_avg', 'item_cnt_month_shop_category_avg', 'item_cnt_month_shop_type_avg', \
                 'item_cnt_month_shop_subtype_avg', 'item_cnt_month_item_city_avg'\
                ]

In [7]:
train_test = average_lag(train_test, group_keys, lag_nums, feature_names)

In [9]:
train_test.tail()

,date_block_num,shop_id,item_id,item_cnt_month,city_code,item_category_id,type_code,subtype_code,item_cnt_month_lag_1,item_cnt_month_lag_2,...,item_cnt_month_category_avg_lag_3,item_cnt_month_category_avg_lag_6,item_cnt_month_category_avg_lag_12,item_cnt_month_type_avg_lag_1,item_cnt_month_subtype_avg_lag_1,item_cnt_month_city_avg_lag_1,item_cnt_month_shop_category_avg_lag_1,item_cnt_month_shop_type_avg_lag_1,item_cnt_month_shop_subtype_avg_lag_1,item_cnt_month_item_city_avg_lag_1
10913799,33,59,22162,0.0,30,40,11,6,0.0,1.0,...,0.247946,0.226550,NaN,0.196066,0.200956,0.179744,0.109834,0.112623,0.109834,0.0
10913800,33,59,22163,0.0,30,40,11,6,0.0,0.0,...,NaN,NaN,NaN,0.196066,0.200956,0.179744,0.109834,0.112623,0.109834,0.0
10913801,33,59,22164,0.0,30,37,11,1,0.0,0.0,...,0.156834,0.189429,NaN,0.196066,0.201937,0.179744,0.131965,0.112623,0.131965,0.0
10913802,33,59,22166,0.0,30,54,12,63,0.0,0.0,...,0.152907,0.148361,0.170513,0.187504,0.076220,0.179744,0.000000,0.033943,0.000000,0.0
10913803,33,59,22167,0.0,30,49,12,41,0.0,0.0,...,0.225480,0.206987,0.221778,0.187504,0.164492,0.179744,0.024390,0.033943,0.024390,0.0


In [10]:
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,revenue
0,2013-02-01,0,59,22154,999.00,1.0,999.00
1,2013-03-01,0,25,2552,899.00,1.0,899.00
2,2013-05-01,0,25,2552,899.00,-1.0,-899.00
3,2013-06-01,0,25,2554,1709.05,1.0,1709.05
4,2013-01-15,0,25,2555,1099.00,1.0,1099.00


In [12]:
## trend fature
group = sales_train.groupby(['item_id']).agg({'item_price': ['mean']})
group.columns = ['item_price_total_avg']
group.reset_index(inplace=True)
train_test = pd.merge(train_test, group, on=['item_id'], how='left')

group = sales_train.groupby(['item_id', 'date_block_num']).agg({'item_price': ['mean']})
group.columns = ['item_price_month_avg']
group.reset_index(inplace=True)
train_test = pd.merge(train_test, group, on=['item_id', 'date_block_num'], how='left')

lag_nums = [1, 2, 3, 4, 5, 6]
train_test = lag_feature(train_test, lag_nums, 'item_price_month_avg')

for i in lag_nums:
    train_test['delta_item_price_lag_' + str(i)] = (train_test['item_price_month_avg_' + str(i)] - train_test['item_price_total_avg']) \
                                                        / train_test['item_price_total_avg']

def select_trend(row):
    for i in lag_nums:
        if row['delta_item_price_lag_' + str(i)]:
            return row['delta_item_price_lag_' + str(i)]
        return 0
train_test['delta_item_price_lag'] = train_test.apply(select_trend, axis=1)
train_test['delta_item_price_lag'] = train_test['delta_item_price_lag'].astypie(np.float64)

features_to_drop = ['item_price_total_avg', 'item_price_month_avg']

for i in lag_nums:
    features_to_drop.append(['item_price_month_avg_' + str(i), 'delta_item_price_lag_' + str(i)])

train_test.drop(features_to_drop, axis=1, inplace=True)


MemoryError: 

In [ ]:
group = sales_train.groupby(['shop_id', 'date_block_num']).agg({'revenue': ['sum']})
group.columns = ['shop_month_revenue']
group.reset_index(inplace=True)
train_test = pd.merge(train_test, group, on=['shop_id', 'date_block_num'], how='left')

group = sales_train.groupby(['date_block_num']).agg({'shop_month_revenue': ['mean']})
group.columns = ['shop_monrh_avg_revenue']
group.reset_index(inplace=True)
train_test = pd.merge(train_test, group, on=['date_block_num'], how='left')


train_test['delta_shop_month_revenue'] = (train_test['shop_month_revenue'] - train_test['shop_month_avg_revenue'])
train_test = lag_feature(train_test, [1], 'delta_shop_month_revenue')

train_test.drop(['shop_month_revenue', 'shop_month_avg_'])